# 데이터 로드 및 인코딩

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [4]:
X = train.iloc[:, 1:-1]
target = test.iloc[:, 1:]

In [5]:
X = np.array(X).reshape(-1, 8, 4, 1)
target = np.array(target).reshape(-1, 8, 4, 1)

In [6]:
ohe = OneHotEncoder(sparse = False)
y = ohe.fit_transform(train[['target']])

In [9]:
skf = StratifiedKFold(n_splits = 10, random_state = 1, shuffle = True)

# 모델링

In [10]:
es = EarlyStopping(monitor = 'val_acc', patience = 50, mode = 'max', verbose = 0)
rlrp = ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, mode='max')

# 콜백함수 중 ReduceLROnPlateau를 새롭게 알게되어 써봤습니다. factor는 0.1보다는 0.2가 더 좋은 성능을 보였습니다

In [ ]:
cnn_acc = []
cnn_pred = np.zeros((target.shape[0], 4))
for i, (tr_idx, val_idx) in enumerate(skf.split(X, train.target)) :
    print(f'{i + 1} Fold Training.....')
    tr_x, tr_y = X[tr_idx], y[tr_idx]
    val_x, val_y = X[val_idx], y[val_idx]
    
    ### CNN 모델
    cnn = Sequential()
    cnn.add(Conv2D(64, (2, 4), padding = "same", activation = 'elu', input_shape = (8, 4, 1))) # 정사각필터보다 직사각필터가 좋은 성능
    cnn.add(BatchNormalization())
    
    cnn.add(Conv2D(32, (2, 4), padding = "same", activation = 'elu'))
    cnn.add(BatchNormalization())
    
    cnn.add(Conv2D(16, (3, 4), padding = "same", activation = 'elu'))
    cnn.add(BatchNormalization())
    
    cnn.add(Conv2D(32, (3, 4), padding = "same", activation = 'elu'))
    
    cnn.add(BatchNormalization())
    cnn.add(GlobalAveragePooling2D())
    
    cnn.add(Dense(16, activation = 'relu'))
    cnn.add(Dense(4, activation = 'softmax'))

    ### ModelCheckPoint Fold마다 갱신
    mc = ModelCheckpoint(f'model_{i + 1}.h5', save_best_only = True, monitor = 'val_acc', mode = 'max', verbose = 0)
   
    ### 모델 compile
    cnn.compile(optimizer = RMSprop(lr=0.04), loss = 'categorical_crossentropy', metrics = ['acc'])

    cnn.fit(tr_x, tr_y, validation_data = (val_x, val_y), epochs = 100, batch_size = 32, callbacks = [es, mc, rlrp], verbose = 0)

    ### 최고 성능 기록 모델 Load
    best = load_model(f'model_{i + 1}.h5')
    ### validation predict
    val_pred = best.predict(val_x)
    ### 확률값 중 최대값을 클래스로 매칭
    val_cls = np.argmax(val_pred, axis = 1)
    ### Fold별 정확도 산출
    fold_cnn_acc = accuracy_score(np.argmax(val_y, axis = 1), val_cls)
    cnn_acc.append(fold_cnn_acc)
    print(f'{i + 1} Fold ACC of CNN = {fold_cnn_acc}\n')

    ### Fold별 test 데이터에 대한 예측값 생성 및 앙상블
    fold_pred = best.predict(target) / skf.n_splits
    cnn_pred += fold_pred

In [ ]:
np.mean(cnn_acc)

# 정확도들의 평균이 0.8972로 나왔습니다.

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = np.argmax(cnn_pred, axis = 1)

submission

submission.to_csv('submission.csv',index=False)